This project is based on the available kaggle competition - Detection of toxic comment.  
link: https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge

Let's import necessary packages first.

In [4]:
import os
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout,Conv1D,Flatten,Concatenate
from keras.models import Model
from keras import optimizers
from keras.callbacks import ModelCheckpoint
#Loading the data set - training data.
import sys, os, re, csv, codecs, numpy as np, pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from gensim.models import KeyedVectors
from sklearn.preprocessing import Imputer
from nltk.tokenize import wordpunct_tokenize
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegressionCV
from sklearn.svm import LinearSVC, NuSVC
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

C:\Users\aas3n17\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\aas3n17\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
#dataset=pd.read_csv('datasets/tow_classes_tweet.csv', encoding='utf-8', usecols=['sentiment', 'txt'])
dataset=pd.read_csv('C:/Users/aas3n17/Desktop/Organized/datasets/Ar_tweet_cleaned_final.csv', encoding='utf-8', usecols=['sentiment', 'txt'])
train, test = train_test_split(dataset, test_size=0.1)
train.head()
#prints all the categories

,sentiment,txt
685,0,مايستحون
1252,0,انا لاحظت بلبنانين انهم ماعندهم روح الكوميديا...
1196,1,﻿ان اله لا يتخل عن عباده ابدا
276,0,خبر عاجل يمنع لجميع المجانين التقدم الي سوبر ...
1189,1,﻿انبسط يا عم و افرح


In [7]:
# Little preprocessing required
sentences_train = train["txt"].fillna("_na_").values
classes = ["sentiment"]
y = train[classes].values
sentences_test = test["txt"].fillna("_na_").values

In [22]:
# Embedding parameter set
embed_size = 100 # how big is each word vector
max_features = 10000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 50 # max number of words in a comment to use

In [23]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(sentences_train))
tokens_train = tokenizer.texts_to_sequences(sentences_train)
tokens_test = tokenizer.texts_to_sequences(sentences_test)
X_train = pad_sequences(tokens_train, maxlen=maxlen)
X_test = pad_sequences(tokens_test, maxlen=maxlen)

# Create Keras Model

In [24]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
x = LSTM(4, return_sequences=True, dropout=0.2, recurrent_dropout=0.1)(x)
x = Conv1D(16,4,activation='relu')(x)
x = Flatten()(x)
x = Dense(100, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer=optimizers.rmsprop(lr = 0.001,decay = 1e-06), metrics=['accuracy'])
filepath="Weights/weights-improvement.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 50, 100)           1000000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 4)             1680      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 47, 16)            272       
_________________________________________________________________
flatten_3 (Flatten)          (None, 752)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               75300     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
__________

Fit the defined model onto the data:

In [ ]:
model.fit(X_train, y, batch_size=32, epochs=5,callbacks=callbacks_list, verbose=1, validation_split=0.2)

Train on 1404 samples, validate on 351 samples
Epoch 1/5
 448/1404 [========>.....................] - ETA: 4s - loss: 0.6949 - acc: 0.4621

# Select from saved weights as per choice and predict response

In [18]:
from keras.models import load_model
saved_model = load_model('Weights/weights-improvement.hdf5')
y_pred = saved_model.predict(X_test)

In [20]:
y_pred=y_pred.round()
y_test=test.sentiment

In [21]:
        f1_score = metrics.f1_score(y_test, y_pred)

        # macro accuracy (macro average)
        macc = metrics.f1_score(y_test, y_pred, pos_label=None, average='macro')

        # precision and recall
        recall = metrics.recall_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)

        results = [macc, f1_score, precision, recall]
        print(results)

[0.8342494714587738, 0.813953488372093, 0.8235294117647058, 0.8045977011494253]
